<a href="https://colab.research.google.com/github/arjasc5231/Lingometer/blob/speaker_verification/speaker_verification/development.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install import_ipynb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 31.9 MB/s 


In [3]:
%cd /content/drive/MyDrive/team_malmungchi/colab/speaker_verification/code
!ls

/content/drive/.shortcut-targets-by-id/1rEYIox5uYM9uP2spDttCYCqtt2GvSbn1/2022 기술 혁신 실험/colab/speaker_verification/code
batcher.ipynb	   eval_metrics.ipynb  network.ipynb
constants.ipynb    fitter.ipynb        preprocess.ipynb
development.ipynb  loss.ipynb	       validation.ipynb


In [4]:
import os
import sys
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import SGD

#sys.path.append("/content/drive/MyDrive/team_malmungchi/colab/speaker_verification/code")
import import_ipynb
from constants import CHECKPOINTS_DIR, NUM_FRAME, NUM_FBANK
import batcher
import network
import loss
import fitter
from validation import validate

importing Jupyter notebook from constants.ipynb
importing Jupyter notebook from batcher.ipynb
importing Jupyter notebook from network.ipynb
importing Jupyter notebook from loss.ipynb
importing Jupyter notebook from fitter.ipynb
importing Jupyter notebook from validation.ipynb
importing Jupyter notebook from eval_metrics.ipynb


In [5]:
 def train(checkpoint_dir, batcher_name, model_name, loss_name, fit_name):
    
    # Batcher 로드
    Batcher = batcher.get_batcher(batcher_name, None, '../data/simMat_online_batcher')
    #Batcher = batcher.get_batcher(batcher_name, None, '../data/simMat_batcher')
    #Batcher = batcher.get_batcher(batcher_name, None, '../data/naive_batcher')
    num_speaker= Batcher.num_speaker

    # optimizer, loss 객체 생성
    optimizer = tf.optimizers.Adam(learning_rate=0.001)
    Loss = loss.get_loss(loss_name)
    
    # 모델 생성
    Model = network.get_network(model_name, num_speaker)
    Model.summary()
    #Model.compile(optimizer='adam', loss=Loss, metrics=['accuracy']) # 이것도 metric이 경우마다 달라질 수 있겠다. fitter 안으로 넣자.
    Model.compile(optimizer='adam', loss=Loss) # simMat은 accuracy가 없음
    
    # 에폭 설정
    initial_epoch, max_epoch = 0, 10 # validation 테스트를 위해 1로 설정 
    
    """
    # 체크포인트 로드. TODO : name별로 다른 체크포인트 폴더 만들기.
    ensures_dir(CHECKPOINTS_DIR)
    pre_training_checkpoint = load_best_checkpoint(CHECKPOINTS_DIR)
    if pre_training_checkpoint:
        initial_epoch = int(pre_training_checkpoint.split('/')[-1].split('.')[0].split('_')[-1])
        Molde.m.load_weights(pre_training_checkpoint)  # latest one.
    
    # 체크포인트 객체 생성
    checkpoint_name = dsm.m.name + '_checkpoint'
    checkpoint_filename = os.path.join(CHECKPOINTS_DIR, checkpoint_name + '_{epoch}.h5')
    checkpoint = ModelCheckpoint(monitor='val_accuracy', filepath=checkpoint_filename, save_best_only=True)
    
    # early_stopping 객체 생성
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.001, patience=20, verbose=1, mode='max')

    # reduce_lr 객체 생성
    reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=10, min_lr=0.0001, verbose=1)
    """
    
    # train
    fitter.fit(fit_name, Model, Batcher, loss, initial_epoch, max_epoch)
    
    # validation
    eer = validate(Model)
    print(eer)
    # eer함수가 단위가 1이 max인듯. (eer=0.13->eer=13%)
    
    

In [6]:
#train(None, 'naive_batcher', 'naive_model', 'cross_entropy', 'naive_fit')

#train(None, 'simMat_batcher', 'naive_model', 'simMat_loss', 'simMat_fit')

train(None, 'simMat_online_batcher', 'naive_model', 'simMat_loss', 'simMat_online_fit')
# 10000 batch (1000steps_per_epoch*10epoch)만큼 학습시킨 결과 loss 31.6347 -> 26.7269,  eer 0.10078124999999999 효과좋다! offline에 비해 overfitting도 안일어난다!
# 50000 batch (1000*50) 만큼 학습시킨 결과, loss 31.1683 -> 24.9756, eer 0.13515625
# 100000 batch (1000*100) 만큼 학습시킨 결과, loss 30.9971 -> 24.7394, eer 0.15000000000000002. 흐음... 오버피팅인가. 오버피팅이 발생할 수가 있나? 데이터셋을 키워보면 좀 해결되려나
# 1000 batch (1000*1)만큼 학습시킨 결과 eer 0.16328125
# 5000 batch (1000*5)만큼 학습시킨 결과 eer 0.11328125000000003
# 생각보다 overfitting의 영향이 큰데 실험할 떄 epoch을 어떻게 설정하는게 맞을까




load exist data
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 126, 126, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 31, 31, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 29, 29, 1